In [27]:
import pandas as pd
import numpy as np
from functools import reduce

In [28]:
pd.set_option('display.max_rows', 500)

In [29]:
PROJECT_ID = 'hawkfish-prod-0c4ce6d0'

In [30]:
## Battery of tests for PII
table_name = 'scratch.civis_selected'
fields_to_test_missingness = ['vb_email', 
                              'vb_voterbase_phone',
                              'vb_voterbase_age',
                              'vb_voterbase_gender',
                              'gen_band',
                              'income_band',
                              'vb_voterbase_race',
                              ]
group_by_field = 'vb_vf_reg_cass_state'

In [31]:
def run_sql(sql_query):
    df = pd.io.gbq.read_gbq(sql_query, project_id=PROJECT_ID, dialect='standard')
    return df

In [32]:
def row_count(table_name):
    """
    count total number of rows in a table """
    sql_row_count = 'select count(*) from {0}'.format(table_name)
    df_row_count = run_sql(sql_row_count)
    return df_row_count.iloc[0,0]
    

In [33]:
def category_count(group_by_field):
    """
    counts number of observations within each category """
    sql_query = """\
        select count(*) as n_obs_{1}, {1} \
         from {0} \
         group by {1} order by {1}
     """.format(table_name, group_by_field)
    return run_sql(sql_query)



In [34]:
def avail_count(field_name):
    sql_query = """\
     select count(*) as n_avail_{1}, {2} \
     from {0} \
     where not (coalesce(cast({1} as string), '') like '' or coalesce(cast({1} as string), '') like 'Unknown') \
     group by {2} order by {2}
     """.format(table_name, field_name, group_by_field)
    return run_sql(sql_query)

In [35]:
avail_list = []
for x in fields_to_test_missingness:
    print('processing {0}\n'.format(x))
    temp_df = avail_count(x)
    avail_list.append(temp_df)

processing vb_email



Downloading: 100%|██████████| 52/52 [00:00<00:00, 373.72rows/s]


processing vb_voterbase_phone



Downloading: 100%|██████████| 55/55 [00:00<00:00, 367.54rows/s]


processing vb_voterbase_age



...

Downloading: 100%|██████████| 57/57 [00:00<00:00, 356.18rows/s]


processing income_band



Downloading: 100%|██████████| 57/57 [00:00<00:00, 387.71rows/s]


processing vb_voterbase_race



Downloading: 100%|██████████| 57/57 [00:00<00:00, 410.01rows/s]


In [36]:
category_count_df = category_count(group_by_field)

Downloading: 100%|██████████| 57/57 [00:00<00:00, 404.49rows/s]


In [37]:
category_count_df = category_count(group_by_field)

Downloading: 100%|██████████| 57/57 [00:00<00:00, 355.08rows/s]


In [39]:
df_avail = reduce(lambda  left,right: pd.merge(left,right,on=[group_by_field],
                                            how='outer'), [category_count_df] + avail_list).fillna(0)


In [40]:
nobs_col = 'n_obs_' + group_by_field
for x in fields_to_test_missingness:
    col_old = 'n_avail_' + x
    col_new = 'frac_avail_' + x
    df_avail[col_new] = df_avail[col_old] / df_avail[nobs_col]
    

In [41]:
df_avail

,n_obs_vb_vf_reg_cass_state,vb_vf_reg_cass_state,n_avail_vb_email,n_avail_vb_voterbase_phone,n_avail_vb_voterbase_age,n_avail_vb_voterbase_gender,n_avail_gen_band,n_avail_income_band,n_avail_vb_voterbase_race,frac_avail_vb_email,frac_avail_vb_voterbase_phone,frac_avail_vb_voterbase_age,frac_avail_vb_voterbase_gender,frac_avail_gen_band,frac_avail_income_band,frac_avail_vb_voterbase_race
0,62181930,0,29560553.0,39694182.0,44173628,56007728,62181930,62181930,62181930,0.475388,0.638356,0.710393,0.900707,1.0,1.0,1.0
1,5,AA,0.0,2.0,5,5,5,5,5,0.000000,0.400000,1.000000,1.000000,1.0,1.0,1.0
2,42,AE,0.0,5.0,38,40,42,42,42,0.000000,0.119048,0.904762,0.952381,1.0,1.0,1.0
3,532825,AK,47978.0,203039.0,397625,532541,532825,532825,532825,0.090045,0.381061,0.746258,0.999467,1.0,1.0,1.0
4,3276454,AL,1115396.0,2272105.0,3276238,3275273,3276454,3276454,3276454,0.340428,0.693465,0.999934,0.999640,1.0,1.0,1.0
5,14,AP,0.0,1.0,11,14,14,14,14,0.000000,0.071429,0.785714,1.000000,1.0,1.0,1.0
6,1697668,AR,602510.0,1051413.0,1697647,1534992,1697668,1697668,1697668,0.354904,0.619328,0.999988,0.904177,1.0,1.0,1.0
7,4025210,AZ,1204486.0,2438268.0,4019222,3691725,4025210,4025210,4025210,0.299236,0.605749,0.998512,0.917151,1.0,1.0,1.0
8,19760977,CA,6072047.0,12773004.0,19755503,18097393,19760977,19760977,19760977,0.307275,0.646375,0.999723,0.915815,1.0,1.0,1.0
9,3767063,CO,1213707.0,2310943.0,3767063,3754369,3767063,3767063,3767063,0.322189,0.613460,1.000000,0.996630,1.0,1.0,1.0


In [48]:
nobs_cutoff_field = 'n_obs_' + group_by_field
df_avail[group_by_field] = df_avail[group_by_field].astype(str)
df_avail_publish = df_avail.sort_values(by=group_by_field, ascending=False).loc[df_avail[nobs_cutoff_field] >= 5000]
#df_avail_publish = df_avail.sort_values(by=group_by_field, ascending=False)

In [50]:
df_avail_publish.to_gbq('scratch.zzz_df_avail_publish', project_id=PROJECT_ID, if_exists='replace')

1it [00:02,  2.69s/it]


In [53]:
age_dist = category_count('gen_band')

Downloading: 100%|██████████| 7/7 [00:00<00:00, 52.51rows/s]


In [60]:
age_dist['n_obs_gen_band'] = age_dist['n_obs_gen_band']/1e6

In [61]:
age_dist

,n_obs_gen_band,gen_band
0,21.493020,1.Gen Z
1,56.175124,2.Millennial
2,61.985204,3.Gen X
3,76.841003,4.Baby Boomers
4,23.211091,5.The Silent Generation
5,3.118988,6.The Greatest Generation
6,19.239219,7.Unknown
